## Predict Test

In [1]:
import numpy as np
import torch
import torch.nn.functional as F
import sys
import os
import csv
import re

from tqdm import tqdm
from torch.utils.data import DataLoader
from transformers import AutoModelForSequenceClassification, AutoTokenizer
from kiwipiepy import Kiwi

sys.path.append(os.path.abspath(os.path.join(os.getcwd(), os.pardir)))

from NLP.etc_plugin import check_exists_cuda
from NLP.model_plugin import set_seed, prepare_data_and_categories, \
    define_datasets, reshape_to_1d, show_test_evaluation, MODEL_NAME

In [2]:
def aspect_evaluate_tests(device, main_category, test_aspect_klue_sets):
    length = len(test_aspect_klue_sets)

    infers = [[] for i in range(length)]
    infer_labels = [[] for i in range(length)]
    
    for i in range(length):
        print("=============")
        print(i+1, ") Aspect Test ...")
        BEST_MODEL_NAME = f'./../NLP/MODEL/{main_category}/model_aspect_' + str(i) + "_best"
        
        best_model = AutoModelForSequenceClassification.from_pretrained(BEST_MODEL_NAME)
        best_model.to(device) # model을 GPU로 이동
        
        dataloader = DataLoader(test_aspect_klue_sets[i], batch_size=4, shuffle=False)
    
        best_model.eval()
        output_pred = []
        output_prob = []
        labels = []
    
        for data in tqdm(dataloader):
            with torch.no_grad():
                outputs = best_model(
                    input_ids=data['input_ids'].to(device), # data를 GPU로 이동
                    attention_mask=data['attention_mask'].to(device),
                    token_type_ids=data['token_type_ids'].to(device)
                )
            logits = outputs[0]
            prob = F.softmax(logits, dim=-1).detach().cpu().numpy()
            logits = logits.detach().cpu().numpy()
            result = np.argmax(logits, axis=-1)
            labels.append(data['label'].tolist())
    
            output_pred.append(result)
            output_prob.append(prob)
    
        pred_answer, output_prob = np.concatenate(output_pred).tolist(), np.concatenate(output_prob, axis=0).tolist()
        
        infers[i].extend(pred_answer)
        infer_labels[i].extend(labels)

    return infers, infer_labels

In [3]:
def sentiment_evaluate_tests(device, main_category, test_sentiment_klue_sets):
    print("=============")
    print("Sentiment Test ...")
    BEST_MODEL_NAME = f'./../NLP/MODEL/{main_category}/model_sentiment_best'
    
    best_model = AutoModelForSequenceClassification.from_pretrained(BEST_MODEL_NAME)
    best_model.to(device)
    
    dataloader = DataLoader(test_sentiment_klue_sets, batch_size=4, shuffle=False)
    
    best_model.eval()
    output_pred = []
    output_prob = []
    labels = []
    
    for data in tqdm(dataloader):
        with torch.no_grad():
            outputs = best_model(
                input_ids=data['input_ids'].to(device),
                attention_mask=data['attention_mask'].to(device),
                token_type_ids=data['token_type_ids'].to(device)
            )
        logits = outputs[0]
        prob = F.softmax(logits, dim=-1).detach().cpu().numpy()
        logits = logits.detach().cpu().numpy()
        result = np.argmax(logits, axis=-1)
        labels.append(data['label'].tolist())
    
        output_pred.append(result)
        output_prob.append(prob)
    
    pred_answer, output_prob = np.concatenate(output_pred).tolist(), np.concatenate(output_prob, axis=0).tolist()
    
    infers = [pred_answer]
    infer_labels = [labels]

    return infers, infer_labels

## Save Prediction Results

In [4]:
def save_asp_prediction_results(main_category, category_with_original_aspects \
                            , test_ASP_datas, test_ASP_labels, asp_infers):
    """
    BEST 모델에 aspect test 데이터셋을 넣어 예측한 값, 정답 및 리뷰 원문을 csv로 저장하는 함수
    """
    output_file_path = f'./csv_results/{main_category}/'
    output_pred_file_name = 'test_asp_prediction_results.csv'
    output_corr_file_name = 'test_asp_correct_results.csv'
    os.makedirs(output_file_path, exist_ok=True)

    pattern = re.compile(fr'{main_category}#([^\s]+)')

    ### 속성(Aspect) ###
    asp_data_list = [
        ['sentence', 'prediction', 'correct'], # Column Title
    ]

    for i in range(0, len(test_ASP_datas[0])):
        prediction_aspects = []
        correct_aspects = []
        for j in range(0, len(category_with_original_aspects)):
            if asp_infers[j][i]:
                prediction_aspects.append(category_with_original_aspects[j])
            if test_ASP_labels[j][i]:
                correct_aspects.append(category_with_original_aspects[j])
        asp_data_list.append([test_ASP_datas[0][i], prediction_aspects, correct_aspects])

    # Unnesting Prediction
    transformed_asp_prediction_data_list = [
        ['main_category', 'id', 'sentence', 'prediction'],
    ]

    for index, row in enumerate(asp_data_list[1:]):
        sentence, predictions, corrects = row
        for prediction in predictions:
            match = pattern.search(prediction)
            prediction_val = match.group(1)
            transformed_asp_prediction_data_list.append([
                main_category,
                index+1,
                sentence,
                prediction_val
            ])

    with open(output_file_path+output_pred_file_name, mode="w", newline="", encoding="cp949") as file:
        writer = csv.writer(file)
    
        for row in transformed_asp_prediction_data_list:
            writer.writerow(row)

    # Unnesting Correct
    transformed_asp_correct_data_list = [
        ['main_category', 'id', 'sentence', 'correct'],
    ]

    for index, row in enumerate(asp_data_list[1:]):
        sentence, predictions, corrects = row
        for correct in corrects:
            match = pattern.search(correct)
            correct_val = match.group(1)
            transformed_asp_correct_data_list.append([
                main_category,
                index+1,
                sentence,
                correct_val
            ])

    with open(output_file_path+output_corr_file_name, mode="w", newline="", encoding="cp949") as file:
        writer = csv.writer(file)
    
        for row in transformed_asp_correct_data_list:
            writer.writerow(row)

In [5]:
def save_sen_prediction_results(main_category, kiwi \
                                , test_SEN_data, test_SEN_labels, sen_infers):
    """
    BEST 모델에 sentiment test 데이터셋을 넣어 예측한 값, 정답 및 리뷰 원문을 csv로 저장하는 함수
    """
    output_file_path = f'./csv_results/{main_category}/'
    output_file_name = 'test_sen_word_cloud_results.csv'
    os.makedirs(output_file_path, exist_ok=True)

    ### 감성(Sentiment) ###
    sen_word_cloud_data_list = [
        ['noun', 'main_category', 'aspect', 'prediction', 'correct'], # Column Title
    ]

    for i in range(0, len(test_SEN_data)):
        sentence = test_SEN_data[i]
        pattern = re.compile(fr'{main_category}#([^\s]+)')
        match = pattern.search(sentence)
        aspect = match.group(1)
        cleaned_sentence = re.sub(pattern, '', sentence).rstrip()

        tokenized_result = kiwi.tokenize(cleaned_sentence)
        noun_results = [token.form for token in tokenized_result if token.tag in {'NNG', 'NNP'}]

        # Unnesting Nouns
        for noun_result in noun_results:
            sen_word_cloud_data_list.append([noun_result, main_category, aspect, sen_infers[0][i], test_SEN_labels[i]])
    
    with open(output_file_path+output_file_name, mode="w", newline="", encoding="cp949") as file:
        writer = csv.writer(file)
    
        for row in sen_word_cloud_data_list:
            writer.writerow(row)

## Main

In [6]:
if __name__ == '__main__':
    device = torch.device('cuda' if torch.cuda.is_available() else 'cpu')
    check_exists_cuda(device)

    set_seed()
    main_categories = [
        '스킨케어',
        '헤어_바디케어',
        '메이크업_뷰티소품',
        '남성화장품'
    ]

    kiwi = Kiwi()

    for main_category in main_categories:
        print(f"================={main_category}====================")
        trains, validations, tests, category_with_original_aspects = prepare_data_and_categories(main_category)

        tokenizer = AutoTokenizer.from_pretrained(MODEL_NAME)

        test_ASP_datas, test_ASP_labels, test_SEN_data, test_SEN_labels = define_datasets(tests, main_category, category_with_original_aspects)

        # 1차원 변환, aspect는 리스트 반환하고 sentiment는 단일 반환
        test_aspect_klue_sets = reshape_to_1d('aspect', test_ASP_datas, test_ASP_labels, tokenizer)
        test_sentiment_klue_sets = reshape_to_1d('sentiment', test_SEN_data, test_SEN_labels, tokenizer)

        # Aspect model test
        asp_infers, asp_infer_labels = aspect_evaluate_tests(device, main_category, test_aspect_klue_sets)

        # Aspect test scores 출력
        show_test_evaluation('aspect', asp_infers, asp_infer_labels, category_with_original_aspects)

        # Sentiment model test
        sen_infers, sen_infer_labels = sentiment_evaluate_tests(device, main_category, test_sentiment_klue_sets)

        # Sentiment test scores 출력
        show_test_evaluation('sentiment', sen_infers, sen_infer_labels)

        print("================================================")
        
        # 테스트 결과 따로 저장
        save_asp_prediction_results(main_category, category_with_original_aspects \
                                    , test_ASP_datas, test_ASP_labels, asp_infers)
        save_sen_prediction_results(main_category, kiwi \
                                    , test_SEN_data, test_SEN_labels, sen_infers)
        

========cuda========
Device: <class 'torch.cuda.device'>
Count of using GPUs: 2
Current cuda device: 0
=================스킨케어====================
1 ) Aspect Test ...


100%|█████████████████████████████████████████████████████████████████████████████████| 818/818 [00:13<00:00, 61.32it/s]


2 ) Aspect Test ...


100%|█████████████████████████████████████████████████████████████████████████████████| 818/818 [00:13<00:00, 62.03it/s]


3 ) Aspect Test ...


...

20 ) Aspect Test ...


100%|█████████████████████████████████████████████████████████████████████████████████| 238/238 [00:03<00:00, 61.78it/s]


0 th Test.....
Accuracy:  0.9863157894736843
Precision: 0.9894459102902374
Recall: 0.9765625
F1-score: 0.9829619921363041
1 th Test.....
Accuracy:  0.9452631578947368
Precision: 0.8533834586466166
Recall: 0.9458333333333333
F1-score: 0.8972332015810276
2 th Test.....
Accuracy:  0.9989473684210526
Precision: 0.9615384615384616
Recall: 1.0
F1-score: 0.9803921568627451
3 th Test.....
Accuracy:  0.9936842105263158
Precision: 0.9647058823529412
Recall: 0.9647058823529412
F1-score: 0.9647058823529412
4 th Test.....
Accuracy:  0.9789473684210527
Precision: 0.9532374100719424
Recall: 0.9742647058823529
F1-score: 0.9636363636363636
5 th Test.....
Accuracy:  0.9989473684210526
Precision: 0.9787234042553191
Recall: 1.0
F1-score: 0.989247311827957
6 th Test.....
Accuracy:  0.9989473684210526
Precision: 0.972972972972973
Recall: 1.0
F1-score: 0.9863013698630138
7 th Test.....
Accuracy:  0.9894736842105263
Precision: 0.9270833333333334
Recall: 0.967391304347826
F1-score: 0.9468085106382979
8 th Test

100%|█████████████████████████████████████████████████████████████████████████████████| 772/772 [00:12<00:00, 61.37it/s]


0 th Test.....
Accuracy:  0.8959818535320804
Precision: 0.8948891012842947
Recall: 0.8959818535320804
F1-score: 0.8944741476599541
